In [61]:
#reads coordinates from CSV
import pandas as pd

# Read coordinates from CSV
csv_file = '/Users/joshmanto/Downloads/BB/tree-semantics/ROI_CQ_Points.csv'
df = pd.read_csv(csv_file)
df.columns = df.columns.str.strip()  # Strip spaces from column names
df = df[['Longitude (dd)', 'Latitude (dd)']]
df.columns = ['longitude', 'latitude']  # Rename columns for consistency
df.head()


,longitude,latitude
0,107.671197,33.663957
1,107.671197,33.663957
2,107.671589,33.663890
3,107.671980,33.663823
4,107.672437,33.663797


In [62]:
from shapely.geometry import Polygon

# Define AOI using the coordinates from the CSV file
def define_aoi(coordinates):
    coords = list(zip(coordinates['longitude'], coordinates['latitude']))
    AOI = {
        "type": "Polygon",
        "coordinates": [coords]
    }
    return AOI

coordinates = df
AOI = define_aoi(coordinates)
print(AOI)


{'type': 'Polygon', 'coordinates': [[(107.6711967, 33.6639574), (107.6711969, 33.6639573), (107.6715886, 33.6638901), (107.6719803, 33.663823), (107.6724371, 33.6637969), (107.6728939, 33.6637708), (107.6733183, 33.6636628), (107.67373, 33.6634578), (107.6738084, 33.6634187), (107.6738085, 33.6634187), (107.6738086, 33.6634186), (107.6738905, 33.6633816), (107.6738947, 33.6633796), (107.6741443, 33.6632666), (107.6741678, 33.663256), (107.6741999, 33.6631588), (107.6742188, 33.6631017), (107.6742596, 33.6629782), (107.6742668, 33.6629562), (107.6742689, 33.6624107), (107.6742509, 33.6618749), (107.6742396, 33.6615377), (107.6742305, 33.6614625), (107.6742169, 33.661349), (107.6741447, 33.6607464), (107.673951, 33.6602003), (107.673931, 33.660076), (107.6738967, 33.6598633), (107.6738475, 33.6595575), (107.6738235, 33.6594089), (107.6738255, 33.6588907), (107.6738306, 33.6587919), (107.6738607, 33.6582088), (107.6740258, 33.6576637), (107.67418, 33.6572967), (107.6742891, 33.657037), (1

In [63]:
#we calculate the AOA dynamically based on the AOI we just pulled from the CSV file

def calculate_aoa(coordinates, expansion_factor=10):
    min_lon, max_lon = coordinates['longitude'].min(), coordinates['longitude'].max()
    min_lat, max_lat = coordinates['latitude'].min(), coordinates['latitude'].max()
    
    # Expand the bounding box slightly
    min_lon -= expansion_factor
    max_lon += expansion_factor
    min_lat -= expansion_factor
    max_lat += expansion_factor
    
    AOA = {
        "type": "Polygon",
        "coordinates": [[
            [min_lon, min_lat],
            [min_lon, max_lat],
            [max_lon, max_lat],
            [max_lon, min_lat],
            [min_lon, min_lat]
        ]]
    }
    return AOA

AOA = calculate_aoa(coordinates)
print(AOA)


{'type': 'Polygon', 'coordinates': [[[97.4284011, 23.449145], [97.4284011, 43.7209511], [117.7495449, 43.7209511], [117.7495449, 23.449145], [97.4284011, 23.449145]]]}


In [64]:
from shapely.geometry import shape

# Determine if AOI is inside AOA
def is_inside(AOI, AOA):
    smaller_shape = shape(AOI)
    larger_shape = shape(AOA)
    return smaller_shape.within(larger_shape)

result = is_inside(AOI, AOA)
print(result)  # Should print True or False


True


In [44]:
#this one is problematic. We are successful in converting a custom set of AOI and AOA into a JSON file that can be used for subscription. 
#but subscription is detail specific. Like the ID
subscription_desc = {
   "name": "Changqing Region",
   "source": {
       "type": "orders",
       "parameters": {
           "id": "BIOMASS-PROXY_V3.0_10",
           "start_time": "2020-01-15T00:00:00Z",
           "end_time": "2023-04-24T00:00:00Z",
           "geometry": {
               "coordinates": AOI,
           }
       }
   }
}


In [45]:
# View the JSON object
subscription_desc

{'name': 'Changqing Region',
 'source': {'type': 'orders',
  'parameters': {'id': 'BIOMASS-PROXY_V3.0_10',
   'start_time': '2020-01-15T00:00:00Z',
   'end_time': '2023-04-24T00:00:00Z',
   'geometry': {'coordinates': {'type': 'Polygon',
     'coordinates': [[(107.6711967, 33.6639574),
       (107.6711969, 33.6639573),
       (107.6715886, 33.6638901),
       (107.6719803, 33.663823),
       (107.6724371, 33.6637969),
       (107.6728939, 33.6637708),
       (107.6733183, 33.6636628),
       (107.67373, 33.6634578),
       (107.6738084, 33.6634187),
       (107.6738085, 33.6634187),
       (107.6738086, 33.6634186),
       (107.6738905, 33.6633816),
       (107.6738947, 33.6633796),
       (107.6741443, 33.6632666),
       (107.6741678, 33.663256),
       (107.6741999, 33.6631588),
       (107.6742188, 33.6631017),
       (107.6742596, 33.6629782),
       (107.6742668, 33.6629562),
       (107.6742689, 33.6624107),
       (107.6742509, 33.6618749),
       (107.6742396, 33.6615377),
   

In [69]:
# Set up the API key and endpoint
os.environ['PL_API_KEY'] = 'PLAKe8a128a493104644888a58e5a0b4b780'
PLANET_API_KEY = os.getenv('PL_API_KEY')
BASE_URL = "https://api.planet.com/analytics/"
subscriptions_endpoint = "subscriptions"

# Set up a session and authenticate
session = requests.Session()
session.auth = (PLANET_API_KEY, "")
auth = (PLANET_API_KEY, "")

# Test the connection
request_url_subscription = BASE_URL + subscriptions_endpoint
res = session.get(request_url_subscription)
print(res.status_code)  # Should print 200 if the connection is successful
print("if 200 shows then authentication is successful")
print(res.text)  # Print the response body


200
if 200 shows then authentication is successful
{"data":[],"links":[{"href":"https://api.planet.com/analytics/subscriptions","rel":"self","type":"application/json"}]}


Attempt to download a single image 4 bands of the bounding box we have. We can access more information about the kinds of ITEMS and ASSETS that planetscope has https://developers.planet.com/docs/apis/data/items-assets/


In [66]:
# feeds_endpoint = "feeds"
# 
# # Construct the URL for the HTTP request 
# #(Planet Analytics API base URL + desired endpoint)
# request_url_feeds = BASE_URL + feeds_endpoint
# response = requests.get(request_url_feeds, auth=auth)
# print(response)
# 
# 
# # Get a feed ID the user has access to
# feed_data = response.json()
# if 'data' in feed_data and len(feed_data['data']) > 0:
#     feed_id = feed_data['data'][0]['id']
#     print('feed_id: {}'.format(feed_id))
# 
#     # URL to request a single feed
#     single_feed_url = request_url_feeds + "/" + feed_id
#     print('single_feed_url: {}'.format(single_feed_url))
# 
#     # Get the single Feed from our GET request response
#     single_feed = requests.get(single_feed_url, auth=auth).json()
#     print(json.dumps(single_feed, indent=1))
# else:
#     print("No feeds available or access denied.")
# 


<Response [200]>
No feeds available or access denied.


In [67]:
# response = requests.get(f"{BASE_URL}/feeds/", auth=auth)
# feeds = response.json()['data']
# for feed in feeds:
#     print(feed['id'], feed['title'])


In [50]:
# #attempt to create a subscription 
# 
# import json
# 
# # Set content type to json
# headers = {'content-type': 'application/json'}
# 
# # Create a subscription
# def subscribe_pv(subscription_desc, auth, headers):
#     response = requests.post(BASE_URL, data=json.dumps(subscription_desc), auth=auth, headers=headers)
#     print(response)
#     if response.status_code == 201:  # 201 Created
#         subscription_id = response.json()['id']
#         print(subscription_id)
#         subscription_url = BASE_URL + '/' + subscription_id
#         return subscription_url
#     else:
#         print("Failed to create subscription")
#         print(response.text)
#         return None
# 
# pv_subscription = subscribe_pv(subscription_desc, auth, headers)
# if pv_subscription:
#     print(f"Subscription created: {pv_subscription}")
# else:
#     print("Subscription creation failed")

<Response [422]>
Failed to create subscription
{"code":602,"message":"endTime in body is required"}
Subscription creation failed
